# STEP 0: Install dependencies

In [1]:
pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [3]:
os.environ["GOOGLE_API_KEY"]= getpass.getpass("Enter your api key")

Enter your api key··········


# STEP 2: Initialize Gemini 2.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-2.5-flash", temperature =0.2)

# STEP 3: Node to ask user for symptom

In [5]:
def get_symptom(state: dict) -> dict:
  symptom = input("Welcome to AS Hospital, Please enter your symptom:")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [6]:
def classify_symptom(state: dict) -> dict:
  prompt =(
      "You are a helpful medical assistant, classify the symptom below into one of the categories \n "
      "-General\n -Emergency\n -Nerve Problem\n"
      f"Symptom: {state['symptom']}\n"
      "Respond only with one word: General, Emergency or Nerve Problem"
      "#Example: input : I have a headache, output: General"
  )

  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM Classifies the symptom as : {category}")
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [7]:
def symptom_router(state: dict) -> dict:
  cat = state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency" in cat:
    return "emergency"
  elif "nerve problem" in cat:
    return "nerve problem"
  else:
    return "general"

# STEP 6: Category-specific response nodes


In [8]:
def general_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : seems general : directing you to general ward for consulting a doctor"
  return state

def emergency_node(state:dict) -> dict:
  state["answer"] =f" ' {state['symptom']}': It is a Medical Emergency : seeking immediate help"
  return state

def nerve_problem_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}: seems like a medical health issue: talk to our counsellor"
  return state

# STEP 7: Build LangGraph


In [9]:
builder = StateGraph(dict)

#define the nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("nerve_problem", nerve_problem_node)

builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "nerve_problem": "nerve_problem"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("nerve_problem", END)

# STEP 8: Compile and invoke the graph


In [10]:
graph = builder.compile()


In [11]:
final_state = graph.invoke({})
print("final Output \n")
print(final_state["answer"])

Welcome to AS Hospital, Please enter your symptom:I can not concentrate
LLM Classifies the symptom as : General
final Output 

 'I can not concentrate' : seems general : directing you to general ward for consulting a doctor
